In [1]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import random
import numpy as np
import torch.nn as nn
import torch
from transformers import AutoModel
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import AdamW, get_linear_schedule_with_warmup
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
# from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
# from ignite.metrics import Accuracy, Loss
# from ignite.engine.engine import Engine, State, Events
# from ignite.handlers import EarlyStopping
# from ignite.contrib.handlers import TensorboardLogger, ProgressBar
# from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from transformers import AutoTokenizer, AutoModel, AdamW
import random
import os
from urllib import request
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from transformers import AutoModel, PreTrainedModel
from transformers import BertModel, BertPreTrainedModel, BertForMaskedLM
from transformers.models.bert.modeling_bert import BertOnlyMLMHead, BertLayer

In [2]:
from sklearn.svm import LinearSVC
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import sys
import argparse
import csv
from scipy.sparse import *
import pandas as pd
import regex 
from sklearn.base import TransformerMixin
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.util import ngrams
from sklearn.model_selection import StratifiedKFold, train_test_split

In [9]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

from transformers import BertConfig, BertTokenizer, BertForMaskedLM,AutoModelForMaskedLM,AutoModel,AutoTokenizer,AutoConfig
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# import xgboost as xgb



In [10]:
def print_statistics(y, y_pred):
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'C': C_range}
    linear_svm = LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'kernel': ['rbf'], 'C': C_range, 'gamma': gamma_range}
    nonlinear_svm = SVC(class_weight=class_ratio)
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'C': C_range}
    log_regr = LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2')
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/svm_model_bin.sav'
    # joblib.dump(svm, filename)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2')
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
    # joblib.dump(regr, filename)


def random_forest(x_train, y_train, x_test, y_test, class_ratio='balanced'):
  print_model_title("Random Forest")
  rf = RandomForestClassifier(n_estimators=400, random_state=11)
  rf.fit(x_train, y_train)
  y_hat = rf.predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost")
  xgb_model =XGBClassifier(max_depth=6, n_estimators=1000)
  xgb_model .fit(x_train, y_train)
  y_hat = xgb_model .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def xg_boost_focal_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Focal")
  xgboster_focal = imb_xgb(special_objective='focal')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost_weighted_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Weighted")
  xgboster_focal = imb_xgb(special_objective='weighted')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = LinearSVC(C=0.1, penalty='l2')
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)


def ensemble_stacked(x_train, y_train, x_test, y_test):
  print_model_title("Ensemble Stacked Classifiers")
  estimators = [ ('lr',LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),('xgb',XGBClassifier(max_depth=16, n_estimators=1000)),('svm_linear',LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),('rf', RandomForestClassifier(n_estimators=10, random_state=42))]
  from sklearn.ensemble import StackingClassifier
  clf = StackingClassifier(
      estimators=estimators )
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
def voting_classifiers(x_train, y_train, x_test, y_test,voting_type='hard'):
  print_model_title("Voting Classifier")
  estimators = [ ('lr',LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),('xgb',XGBClassifier(max_depth=16, n_estimators=1000)),('svm_linear',LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),('rf', RandomForestClassifier(n_estimators=10, random_state=42))]
  from sklearn.ensemble import StackingClassifier
  clf = VotingClassifier(
      estimators=estimators , voting=voting_type)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
def Bagging_Classifier_LR(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier LR")
 
  clf =BaggingClassifier(base_estimator=LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def Bagging_Classifier_SVM(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier SVM")
 
  clf =BaggingClassifier(base_estimator=LinearSVC(C=0.01, class_weight='balanced', penalty='l2'),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


from sklearn.ensemble import GradientBoostingClassifier
def gradient_boosting(x_train, y_train, x_test, y_test):
  print_model_title("Gradient Boosting")
 
  clf =GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=30, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)


In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

In [13]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # print(token_embeddings.shape)
    # token_vecs_cat = []
    # for token in token_embeddings:
    #   cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [23]:
tokenizer= AutoTokenizer.from_pretrained('UBC-NLP/MARBERTv2')
model=AutoModel.from_pretrained(('UBC-NLP/MARBERTv2'),output_hidden_states = True)


Some weights of the model checkpoint at UBC-NLP/MARBERTv2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
train = pd.read_csv('/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Dataset/NADI2022_Subtask2_TRAIN.tsv', sep='\t', lineterminator='\n')
test = pd.read_csv('/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Dataset/NADI2022_Subtask2_DEV.tsv', sep='\t', lineterminator='\n')

In [21]:
train.rename(columns = {'#2_content':'text', '#3_label':'label'}, inplace = True)
test.rename(columns = {'#2_content':'text', '#3_label':'label'}, inplace = True)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['label'] = le.fit_transform(train['label'].values)
le = LabelEncoder()

test['label'] = le.fit_transform(test['label'].values)

In [31]:
target_word_embeddings = []
wordvec_arrays = np.zeros((len(train['text']), 768)) 
i=0
# for i in range(len(tokenized_tweet)):
#     # print(i)
    


for text in train['text'].values:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    def word_vector(tokens, size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              # print(model_w2v[word].shape)
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec
    wordvec_arrays[i,:] = word_vector(list_token_embeddings, 768)
    i=i+1
    # print(len(list_token_embeddings))
    # target_word_embeddings.append(np.array(list_token_embeddings))
    
    # # Find the position 'bank' in list of tokens
    # word_index = tokenized_text.index('bank')
    # # Get the embedding for bank
    # word_embedding = list_token_embeddings[word_index]

    # target_word_embeddings.append(word_embedding)

In [32]:
wordvec_df_new = pd.DataFrame(wordvec_arrays)

In [33]:
target_word_embeddings = []
wordvec_arrays = np.zeros((len(test['text']), 768)) 
i=0
# for i in range(len(tokenized_tweet)):
#     # print(i)
    


for text in test['text'].values:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    def word_vector(tokens, size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              # print(model_w2v[word].shape)
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec
    wordvec_arrays[i,:] = word_vector(list_token_embeddings, 768)
    i=i+1
    # print(len(list_token_embeddings))
    # target_word_embeddings.append(np.array(list_token_embeddings))
    
    # # Find the position 'bank' in list of tokens
    # word_index = tokenized_text.index('bank')
    # # Get the embedding for bank
    # word_embedding = list_token_embeddings[word_index]

    # target_word_embeddings.append(word_embedding)

In [34]:
wordvec_df_test= pd.DataFrame(wordvec_arrays)

In [29]:
# wordvec_df_new.shape

(500, 768)

In [30]:
# train['label'].shape

(1500,)

In [35]:
gradient_boosting(wordvec_df_new,train['label'],wordvec_df_test,test['label'])


   Gradient Boosting

Accuracy: 0.468
Precision: 0.456
Recall: 0.468
F_score: 0.455

              precision    recall  f1-score   support

           0       0.49      0.46      0.47       190
           1       0.34      0.21      0.26       113
           2       0.49      0.62      0.55       197

    accuracy                           0.47       500
   macro avg       0.44      0.43      0.43       500
weighted avg       0.46      0.47      0.45       500



In [36]:
Bagging_Classifier_SVM(wordvec_df_new,train['label'],wordvec_df_test,test['label'])


Bagging Calssifier SVM

Accuracy: 0.564
Precision: 0.554
Recall: 0.564
F_score: 0.554

              precision    recall  f1-score   support

           0       0.58      0.68      0.62       190
           1       0.45      0.30      0.36       113
           2       0.59      0.60      0.60       197

    accuracy                           0.56       500
   macro avg       0.54      0.53      0.53       500
weighted avg       0.55      0.56      0.55       500



In [37]:
Bagging_Classifier_LR(wordvec_df_new,train['label'],wordvec_df_test,test['label'])


Bagging Calssifier LR

Accuracy: 0.526
Precision: 0.533
Recall: 0.526
F_score: 0.529

              precision    recall  f1-score   support

           0       0.56      0.55      0.55       190
           1       0.38      0.43      0.40       113
           2       0.59      0.56      0.58       197

    accuracy                           0.53       500
   macro avg       0.51      0.51      0.51       500
weighted avg       0.53      0.53      0.53       500



In [39]:
# voting_classifiers(wordvec_df_new,train['label'],wordvec_df_test,test['label'])

In [ ]:
# ensemble_stacked(wordvec_df_new,train['label'],wordvec_df_test,test['label'])

In [ ]:
# xg_boost(wordvec_df_new,train['label'],wordvec_df_test,test['label'])

In [44]:
random_forest(wordvec_df_new,train['label'],wordvec_df_test,test['label'],class_ratio='balanced')


       Random Forest

Accuracy: 0.556
Precision: 0.559
Recall: 0.556
F_score: 0.497

              precision    recall  f1-score   support

           0       0.55      0.69      0.61       190
           1       0.57      0.04      0.07       113
           2       0.56      0.73      0.63       197

    accuracy                           0.56       500
   macro avg       0.56      0.48      0.44       500
weighted avg       0.56      0.56      0.50       500



In [41]:
logistic_regression(wordvec_df_new,train['label'],wordvec_df_test,test['label'],class_ratio='balanced')


 Logistic Regression

Accuracy: 0.528
Precision: 0.543
Recall: 0.528
F_score: 0.532

              precision    recall  f1-score   support

           0       0.55      0.57      0.56       190
           1       0.39      0.49      0.43       113
           2       0.63      0.51      0.56       197

    accuracy                           0.53       500
   macro avg       0.52      0.52      0.52       500
weighted avg       0.54      0.53      0.53       500



In [42]:
linear_svm(wordvec_df_new,train['label'],wordvec_df_test,test['label'],class_ratio='balanced')


          Linear SVM

Accuracy: 0.568
Precision: 0.561
Recall: 0.568
F_score: 0.558

              precision    recall  f1-score   support

           0       0.56      0.67      0.61       190
           1       0.49      0.31      0.38       113
           2       0.60      0.61      0.61       197

    accuracy                           0.57       500
   macro avg       0.55      0.53      0.53       500
weighted avg       0.56      0.57      0.56       500



In [43]:
logistic_regression_grid(wordvec_df_new,train['label'],wordvec_df_test,test['label'],class_ratio='balanced')


 Logistic Regression

Accuracy: 0.582
Precision: 0.601
Recall: 0.582
F_score: 0.589

              precision    recall  f1-score   support

           0       0.67      0.62      0.65       190
           1       0.36      0.45      0.40       113
           2       0.67      0.62      0.64       197

    accuracy                           0.58       500
   macro avg       0.57      0.56      0.56       500
weighted avg       0.60      0.58      0.59       500



In [45]:
nonlinear_svm_grid(wordvec_df_new,train['label'],wordvec_df_test,test['label'],class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.572
Precision: 0.577
Recall: 0.572
F_score: 0.574

              precision    recall  f1-score   support

           0       0.63      0.64      0.63       190
           1       0.38      0.42      0.40       113
           2       0.64      0.60      0.62       197

    accuracy                           0.57       500
   macro avg       0.55      0.55      0.55       500
weighted avg       0.58      0.57      0.57       500



In [46]:
linear_svm_grid(wordvec_df_new,train['label'],wordvec_df_test,test['label'],class_ratio='balanced')


          Linear SVM

Accuracy: 0.610
Precision: 0.605
Recall: 0.610
F_score: 0.607

              precision    recall  f1-score   support

           0       0.66      0.71      0.68       190
           1       0.42      0.38      0.40       113
           2       0.66      0.65      0.65       197

    accuracy                           0.61       500
   macro avg       0.58      0.58      0.58       500
weighted avg       0.60      0.61      0.61       500

